In [1]:
import numpy as np
import tensorflow as tf


import matplotlib.pyplot as plt
tf.logging.set_verbosity(tf.logging.INFO)

In [2]:
%load_ext autoreload
%autoreload 2
import sgld_tf

Instructions for updating:
Use the retry module or similar alternatives.


In [3]:
mnist = tf.contrib.learn.datasets.load_dataset("mnist")
train_data = mnist.train.images # Returns np.array
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
eval_data = mnist.test.images # Returns np.array
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

Instructions for updating:
Please use tf.data.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
cnn = sgld_tf.CNN(train_data, train_labels,
                  eval_data, eval_labels,
                  bs=512, model_dir="/tmp/tf/dec_ksgld1/",
                  decay=.96, decay_interval=50,
                 optimize_method="ksgld",
                 learning_rate=0.001,
                 save_checkpoint_steps=10,
                 keep_checkpoint_max=1000000, 
                 shuffle=True)

INFO:tensorflow:Using config: {'_model_dir': '/tmp/tf/dec_ksgld1/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 10, '_save_checkpoints_secs': None, '_session_config': None, '_keep_checkpoint_max': 1000000, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f10b5ff0cc0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [5]:
train_spec = tf.estimator.TrainSpec(input_fn=cnn.train_input_fn, max_steps=3000)
eval_spec = tf.estimator.EvalSpec(input_fn=cnn.eval_input_fn, throttle_secs=30)

In [6]:
tf.estimator.train_and_evaluate(cnn.estimator, train_spec, eval_spec)

INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 30 secs (eval_spec.throttle_secs) or training is finished.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tf/dec_ksgld1/model.ckpt.
INFO:tensorflow:loss = 2.30125, step = 0
INFO:tensorflow:Saving checkpoints for 11 into /tmp/tf/dec_ksgld1/model.ckpt.
INFO:tensorflow:Saving checkpoints for 21 into /tmp/tf/dec_ksgld1/model.ckpt.
INFO:tensorflow:Saving checkpoints for 31 into /tmp/tf/dec_ksgld1/model.ckpt.
INFO:tensorflow:Saving checkpoints for 41 into /tmp/tf/dec_ksgld1/model.ckpt.
INFO:tensorflow:Saving checkpoints for 51 into /tmp/tf/dec_ksgld1/model.ckpt.
INFO:tensorflow:Saving checkpoints for 61 i

In [7]:
ha = cnn.estimator.predict(input_fn=cnn.eval_input_fn)

In [17]:
cnn.estimator.get_variable_value("global_step")

10000

In [18]:
cnn2 = sgld_tf.CNN(train_data, train_labels,
                  eval_data, eval_labels,
                  bs=512, model_dir="/tmp/tf/sgld_60/",
                 optimize_method="sgld",
                 learning_rate=0.001,
                 save_checkpoint_steps=10,
                 keep_checkpoint_max=1000000, 
                 shuffle=True)

INFO:tensorflow:Using config: {'_model_dir': '/tmp/tf/sgld_60/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 10, '_save_checkpoints_secs': None, '_session_config': None, '_keep_checkpoint_max': 1000000, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f2d8554c198>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [12]:
ha2 = cnn2.estimator.predict(input_fn = cnn2.eval_input_fn, checkpoint_path='/tmp/tf/try_predict/model.ckpt-3000')

In [13]:
next(ha2)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tf/try_predict/model.ckpt-3000


Exception ignored in: <generator object Estimator.predict at 0x7fc772510938>
Traceback (most recent call last):
  File "/home/henripal/anaconda3/lib/python3.6/site-packages/tensorflow/python/estimator/estimator.py", line 511, in predict
    for key, value in six.iteritems(preds_evaluated)
  File "/home/henripal/anaconda3/lib/python3.6/contextlib.py", line 99, in __exit__
    self.gen.throw(type, value, traceback)
  File "/home/henripal/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/ops.py", line 5086, in get_controller
    yield g
  File "/home/henripal/anaconda3/lib/python3.6/contextlib.py", line 99, in __exit__
    self.gen.throw(type, value, traceback)
  File "/home/henripal/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/ops.py", line 4904, in get_controller
    type(default))
AssertionError: Nesting violated for default stack of <class 'tensorflow.python.framework.ops.Graph'> objects


INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


{'classes': 7,
 'probabilities': array([  5.31080696e-05,   5.68719997e-06,   2.26426564e-05,
          1.52783818e-03,   7.97744360e-06,   1.19294476e-04,
          6.53315169e-07,   9.96016800e-01,   2.23172785e-04,
          2.02273391e-03], dtype=float32)}

In [ ]:
cnn